

## steps
- create a telegram bot
- set the token on the code
- run all


## How to use
go to your bot on telegram send the Prompts separated by `__`, (two underscores).
before the separator will be the positive prompt, after that will be the negative prompt

## Mount Google Drive

In [1]:
import os
from google.colab import drive

drivePath = '/content/drive'

if os.path.isdir(drivePath) == False:
    drive.mount(drivePath)

Mounted at /content/drive


## Install Requirements

In [2]:
%pip install --quiet nest-asyncio transformers diffusers python-telegram-bot accelerate triton scipy ftfy spacy xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.4/545.4 kB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [3]:
!pip install torch --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import and Setup Bot + SD

In [4]:
from telegram import Update, Bot
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters
import nest_asyncio
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
import asyncio
import random

model_id = "runwayml/stable-diffusion-v1-5"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")



if os.path.isdir(f'{drivePath}/MyDrive/sd_1-5'):
    pipe = StableDiffusionPipeline.from_pretrained(f'{drivePath}/MyDrive/sd_1-5', torch_dtype=torch.float16, scheduler=scheduler)
else:
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, scheduler=scheduler)
    pipe.save_pretrained(f'{drivePath}/MyDrive/sd_1-5')


pipe = pipe.to("cuda")
pipe.enable_attention_slicing() # to use less memory
pipe.safety_checker = None # to disable NSFW filter

nest_asyncio.apply()
token = "your token"

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


## Setup Handler for bot

In [5]:
default_steps = 20

async def generate(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
  text = update.message.text
  prompt = ''
  neg = ''
  if '__' in text:
    prompt = text.split("__")[0]
    neg = text.split("__")[1]
  else:
    prompt = text
  seed = random.randint(0, 2147483647)
  image = pipe(
    prompt,
    negative_prompt=neg,
    num_inference_steps=default_steps,
    height=512,
    width=512,
    guidance_scale = 8,
    generator = torch.Generator("cuda").manual_seed(seed)
    ).images[0]
  image.save("./image.png")
  await update.message.reply_photo(photo="./image.png")

app = ApplicationBuilder().token(token).build()

app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, generate))

app.run_polling()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 29>:29                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/telegram/ext/_application.py:727 in run_polling          │
│                                                                                                  │
│    724 │   │   def error_callback(exc: TelegramError) -> None:                                   │
│    725 │   │   │   self.create_task(self.process_error(error=exc, update=None))                  │
│    726 │   │                                                                                     │
│ ❱  727 │   │   return self.__run(                                                                │
│    728 │   │   │   updater_coroutine=self.updater.start_polling(                                 │
│    729 │   │   │   │   poll_interval=poll_interval,                                              │
│    730 │   │   │   │   timeout=timeout,                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/telegram/ext/_application.py:937 in __run                │
│                                                                                                  │
│    934 │   │   │   │   │   loop.run_until_complete(self.post_shutdown(self))                     │
│    935 │   │   │   finally:                                                                      │
│    936 │   │   │   │   if close_loop:                                                            │
│ ❱  937 │   │   │   │   │   loop.close()                                                          │
│    938 │                                                                                         │
│    939 │   def create_task(                                                                      │
│    940 │   │   self,                                                                             │
│                                                                                                  │
│ /usr/lib/python3.10/asyncio/unix_events.py:68 in close                                           │
│                                                                                                  │
│     65 │   │   self._signal_handlers = {}                                                        │
│     66 │                                                                                         │
│     67 │   def close(self):                                                                      │
│ ❱   68 │   │   super().close()                                                                   │
│     69 │   │   if not sys.is_finalizing():                                                       │
│     70 │   │   │   for sig in list(self._signal_handlers):                                       │
│     71 │   │   │   │   self.remove_signal_handler(sig)                                           │
│                                                                                                  │
│ /usr/lib/python3.10/asyncio/selector_events.py:84 in close                                       │
│                                                                                                  │
│     81 │                                                                                         │
│     82 │   def close(self):                                                                      │
│     83 │   │   if self.is_running():                                                             │
│ ❱   84 │   │   │   raise RuntimeError("Cannot close a running event loop")                       │
│     85 │   │   if self.is_closed():                        